In [6]:
from cerebralcortex import Kernel
from cerebralcortex.core.datatypes import DataStream
from util.data_helper import gen_phone_battery_data, gen_phone_battery_metadata

In [ ]:
CC = Kernel("/home/md2k/cc_conf/")

In [9]:
class Examples:
    def __init__(self):
        """
        load/set example params/data. This example perform following operations:
            - create sample phone battery data stream
            - perform windowing operation on the stream
            - store windowed data asa new stream
        Args:
            example_name: windowing
        """

        self.setup_example()

        self.basic_datastream_operations()

    def setup_example(self):
        """
        setup required params for the example:
            - create cerebralcortex-kernel object
            - generate sample phone-battery data/metadata
            - create a DataStream object
            - save sample stream using cerebralcortex-kernel.
        """
        # create cerebralcortex object
        self.CC = Kernel("/home/md2k/cc_conf/")

        # sample data params
        self.stream_name="BATTERY--org.md2k.phonesensor--PHONE"
        self.user_id = "00000000-afb8-476e-9872-6472b4e66b68"

        # generate sample phone-battery data/metadata
        data = gen_phone_battery_data(user_id=self.user_id)
        metadata = gen_phone_battery_metadata(stream_name=self.stream_name)

        # create a DataStream object
        ds = DataStream(data, metadata)

        # save sample data using cerebralcortex-kernal.
        # now we have some sample data stored in CerebralCortex format to play with!!!
        self.CC.save_stream(ds)

    def basic_datastream_operations(self):
        """
        This example will window phone battery stream into 1 minutes chunks and take the average of battery level

        """

        streams = self.CC.list_streams()
        print(streams)
        # [{'name': 'BATTERY--org.md2k.phonesensor--PHONE', 'version': 1, 'description': [''], 'metadata_hash':.......]

        stream_names = self.CC.search_stream("battery")
        print(stream_names)
        #['BATTERY--org.md2k.phonesensor--PHONE', 'BATTERY--org.md2k.phonesensor--PHONE-windowed-data']

        # get sample stream data
        ds = self.CC.get_stream(self.stream_name)
        metadata = ds.get_metadata(version=1)
        print(metadata)
        # {'name': 'BATTERY--org.md2k.phonesensor--PHONE', 'version': 1, 'description': [''], 'metadata_ha.........

        ds.show(3,False)
        # +-------------------+--------+-------------+---+-------+------------------------------------+
        # |timestamp          |offset  |battery_level|ver|version|user                                |
        # +-------------------+--------+-------------+---+-------+------------------------------------+
        # |2019-01-09 11:49:28|21600000|92           |1  |1      |00000000-afb8-476e-9872-6472b4e66b68|
        # |2019-01-09 11:49:29|21600000|92           |1  |1      |00000000-afb8-476e-9872-6472b4e66b68|
        # |2019-01-09 11:49:30|21600000|92           |1  |1      |00000000-afb8-476e-9872-6472b4e66b68|
        # +-------------------+--------+-------------+---+-------+------------------------------------+

        ds.filter("battery_level", ">", 97)
        ds.show(3,False)
        # +-------------------+--------+-------------+---+-------+------------------------------------+
        # |timestamp          |offset  |battery_level|ver|version|user                                |
        # +-------------------+--------+-------------+---+-------+------------------------------------+
        # |2019-01-09 11:39:08|21600000|98           |1  |1      |00000000-afb8-476e-9872-6472b4e66b68|
        # |2019-01-09 11:39:09|21600000|98           |1  |1      |00000000-afb8-476e-9872-6472b4e66b68|
        # |2019-01-09 11:39:10|21600000|98           |1  |1      |00000000-afb8-476e-9872-6472b4e66b68|
        # +-------------------+--------+-------------+---+-------+------------------------------------+

        ds.filter_user("00000000-afb8-476e-9872-6472b4e66b68")
        ds.show(3,False)
        # +-------------------+--------+-------------+---+-------+------------------------------------+
        # |timestamp          |offset  |battery_level|ver|version|user                                |
        # +-------------------+--------+-------------+---+-------+------------------------------------+
        # |2019-01-09 11:39:08|21600000|98           |1  |1      |00000000-afb8-476e-9872-6472b4e66b68|
        # |2019-01-09 11:39:09|21600000|98           |1  |1      |00000000-afb8-476e-9872-6472b4e66b68|
        # |2019-01-09 11:39:10|21600000|98           |1  |1      |00000000-afb8-476e-9872-6472b4e66b68|
        # +-------------------+--------+-------------+---+-------+------------------------------------+

        ds.filter_version("1")
        ds.show(3,False)

        # +-------------------+--------+-------------+---+-------+------------------------------------+
        # |timestamp          |offset  |battery_level|ver|version|user                                |
        # +-------------------+--------+-------------+---+-------+------------------------------------+
        # |2019-01-09 11:39:08|21600000|98           |1  |1      |00000000-afb8-476e-9872-6472b4e66b68|
        # |2019-01-09 11:39:09|21600000|98           |1  |1      |00000000-afb8-476e-9872-6472b4e66b68|
        # |2019-01-09 11:39:10|21600000|98           |1  |1      |00000000-afb8-476e-9872-6472b4e66b68|
        # +-------------------+--------+-------------+---+-------+------------------------------------+

        ds.window(windowDuration=60)
        ds.show(3, False)
        # +------------------------------------+------------------------------------------+---------------------------------
        # |user                                |window                                    |collect_list(battery_level)
        # +------------------------------------+------------------------------------------+---------------------------------
        # |00000000-afb8-476e-9872-6472b4e66b68|[2019-01-09 11:35:00, 2019-01-09 11:36:00]|[100, 100, 100, 100, 100.........
        # |00000000-afb8-476e-9872-6472b4e66b68|[2019-01-09 11:38:00, 2019-01-09 11:39:00]|[99, 99, 99, 99, 99, 99, 99,.....
        # |00000000-afb8-476e-9872-6472b4e66b68|[2019-01-09 11:39:00, 2019-01-09 11:40:00]|[98, 98, 98, 98, 98, 98, 98,.....
        # +------------------------------------+------------------------------------------+---------------------------------

        ds.to_pandas()
        print(ds.data)
        # user  ...                                 collect_list(user)
        # 0  00000000-afb8-476e-9872-6472b4e66b68  ...  [00000000-afb8-476e-9872-6472b4e66b68, 0000000...
        # 1  00000000-afb8-476e-9872-6472b4e66b68  ...  [00000000-afb8-476e-9872-6472b4e66b68, 0000000...
        # 2  00000000-afb8-476e-9872-6472b4e66b68  ...  [00000000-afb8-476e-9872-6472b4e66b68, 0000000...
        # 3  00000000-afb8-476e-9872-6472b4e66b68  ...  [00000000-afb8-476e-9872-6472b4e66b68, 0000000...
        # 4  00000000-afb8-476e-9872-6472b4e66b68  ...  [00000000-afb8-476e-9872-6472b4e66b68, 0000000...

In [10]:
Examples()

[{'name': 'accelerometer--org.md2k.phonesensor--phone', 'version': 1, 'description': 'mobile phone accelerometer sensor data.', 'metadata_hash': '95e542cd-718e-358e-bede-8bbdd97f86d6', 'input_streams': [], 'annotations': [], 'data_descriptor': [<cerebralcortex.core.metadata_manager.stream.data_descriptor.DataDescriptor object at 0x7f77b36b8d30>, <cerebralcortex.core.metadata_manager.stream.data_descriptor.DataDescriptor object at 0x7f77b36b8da0>, <cerebralcortex.core.metadata_manager.stream.data_descriptor.DataDescriptor object at 0x7f77b36b8d68>], 'modules': [<cerebralcortex.core.metadata_manager.stream.module_info.ModuleMetadata object at 0x7f77b36b8dd8>]}, {'name': 'accelerometer--org.md2k.phonesensor--phone_gaussian_noise', 'version': 1, 'description': 'Gaussian noise added to the accel sensor stream.', 'metadata_hash': '5d8d152b-5696-3837-8116-1f77a041d51a', 'input_streams': [], 'annotations': [], 'data_descriptor': [<cerebralcortex.core.metadata_manager.stream.data_descriptor.Dat